In [8]:
import pandas as pd
from pandas import DataFrame
import sqlite3
from sqlite3 import Connection
import os
import base64
import requests
from requests.exceptions import HTTPError
from requests import Session
from typing import List
from typing import Tuple

# Get popularity

The goal of this notebook is to add a column to our existing total_song_data column in the sqlite database with popularity ratings for each track.

These ratings are 0-100 based on Spotify's internal scale.

This file also requires a client secret and client id from Spotify, or a one-time access token from Spotify.


In [9]:
# Get client id and secret.
client_id = os.environ.get('CLIENT_ID')
client_secret = os.environ.get('CLIENT_SECRET')

This code will let us get an access token, which expires in one hour.

In [10]:
def request_client_credentials_token(client_id, client_secret, session: Session) -> str:
    url = "https://accounts.spotify.com/api/token"

    # encode using base64 encoding
    auth_header = base64.b64encode(f"{client_id}:{client_secret}" \
                                    .encode("ascii")) \
                                    .decode("ascii")
    headers = {
        "Authorization": f"Basic {auth_header}"
    }

    # Set up the data to include the grant type
    data = {
        "grant_type": "client_credentials"
    }
    response = session.post(url, 
                            data=data, 
                            headers=headers)
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise requests.exceptions.HTTPError(response.text)

This code will get the popularity rankings for 50 songs (this is the batch limit), using the Spotify track ids, which are stored in our database already. It returns a dict with the information for popularity stored under a key called 'popularity'.

In [11]:
def get_popularity(track_ids: List[str], access_token: str, session: Session) -> dict:
    # Set the Authorization header with the access token
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    # Join the track IDs with commas to create the query parameter
    track_ids_query_param = ','.join(track_ids)
    
    # Make a GET request to the Spotify API with the track IDs in the query parameter
    response = requests.get(f'https://api.spotify.com/v1/tracks?ids={track_ids_query_param}', 
                            headers=headers)
    
    # Raise an HTTPError if the response status code indicates an error
    response.raise_for_status() 
    return response.json()['tracks']


Once we have the data, we can insert it using this function, which uses a sqlite3 insert statement.

In [12]:
def insert_popularity(conn: Connection, pops_and_ids: List[Tuple]) -> None:
    cur = conn.cursor()
    
    # Execute an update query for each tuple in the input list, setting the popularity 
    # column to the first value and the Spotify track ID to the second value
    cur.executemany("""
        UPDATE tracks_top_genres
        SET popularity = ?
        WHERE spotify_track_id = ?
    """, pops_and_ids)

    # Commit the changes to the database
    conn.commit()

This next cell creates the additional column. We only need to run it once, and can leave it commented out for now.

In [13]:
# # Add new column for popularity.
# conn = sqlite3.Connection('data/song_data_2.0.db')
# cur = conn.cursor()
# cur.execute("""
# ALTER TABLE tracks_top_genres
# ADD COLUMN popularity REAL
# """)
# conn.commit()

Finally, we can use the previous functions to request popularity for each song using its id, in batches of 50, which is the max allowed. 

At the beginning, we will select the ids from our database that haven't got a matching popularity score yet, and load those ids into memory.

In [14]:
for i in range(5):
    try:
        conn = sqlite3.Connection('data/song_data_2.0.db')

        # Select all Spotify track IDs with a null popularity value
        ids = pd.read_sql("""SELECT spotify_track_id 
                                FROM tracks_top_genres 
                                WHERE popularity IS NULL
                                ORDER BY RANDOM()""", 
                                conn)

        print('Done reordering. Len ids:', len(ids))

        # Create a session object and request an access token
        session = requests.Session()
        access_token: str = request_client_credentials_token(client_id, 
                                                            client_secret, 
                                                            session)

        # Loop through the IDs in groups of 50 and get their popularity values
        for i in range(0, len(ids), 50):
            # Get the current group of IDs as a list
            id_group: List[str] = ids[i : i + 50].spotify_track_id.tolist()

            # Get the popularity values for the current group of IDs
            popularity_group: List[dict] = get_popularity(id_group, access_token, session)
            
            # Unpack the popularity values from the response dics and create a list
            popularity_unpacked: List[int] = [x['popularity'] for x in popularity_group]

            # Raise an error if the length of the ID group and popularity values don't match
            if len(id_group) != len(popularity_unpacked):
                raise RuntimeError("Len of IDs sent and popularity received don't match.")
            
            # Insert the popularity values into the database for the current group of IDs
            insert_popularity(conn, list(zip(popularity_unpacked, id_group)))

        session.close()
        conn.close()
    except TypeError as te:
        print(te)
    except HTTPError as he:
        print(he)

Done reordering. Len ids: 713014


In [ ]:
conn = sqlite3.Connection('data/song_data_2.0.db')
pd.read_sql('SELECT COUNT(*) FROM tracks_top_genres WHERE popularity IS NOT NULL LIMIT 1', conn)

,COUNT(*)
0,127400


References

- https://docs.python.org/3/library/os.html
- https://docs.python.org/3/library/base64.html
- https://developer.spotify.com/documentation/web-playback-sdk
